In [ ]:
from fastapi import FastAPI
from contextlib import asynccontextmanager
from sqlalchemy import create_engine, MetaData
import json
# Charger les informations de connexion
path_postgresql_creds = r"C:\Credentials\postgresql_creds.json"

with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "Oceanography_ML_Project"
schema = "Silver"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
metadata = MetaData(schema=schema)

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Connexion à la base de données
    metadata.reflect(bind=engine)
    # Charger les tables du schéma "Silver"
    app.state.tables = metadata.tables
    yield
    # Fermeture de la connexion à la base de données
    app.state.tables = None

# Création de l'application FastAPI avec le lifespan
app = FastAPI(lifespan=lifespan)

@app.get("/tables")
async def get_tables():
    # Retourner les tables du schéma Silver
    return {"tables": list(app.state.tables.keys())}
